In [1]:
import os

import pandas as pd
import numpy as np

from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

In [2]:
ranks = {}
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))

In [3]:
df = pd.read_csv("../datasets/austin_housing/processed_austin_housing.csv")
df.head()

zipcode  latitude  longitude  propertyTaxRate  garageSpaces  \
0 -4.019137  1.433803   1.362892        -0.265248      0.570097   
1 -4.019137  1.454849   1.379193        -0.265248      0.570097   
2 -4.019137  1.218441   1.638033        -0.265248     -0.909113   
3 -4.019137  1.449066   1.379644        -0.265248      0.570097   
4 -4.019137  1.503275   1.436293        -0.265248     -0.909113   

   hasAssociation  hasGarage    hasSpa   hasView  homeType  ...  \
0        0.945895   0.904299 -0.292941 -0.543043  0.183428  ...   
1        0.945895   0.904299 -0.292941 -0.543043  0.183428  ...   
2        0.945895  -1.105829 -0.292941 -0.543043  0.183428  ...   
3        0.945895   0.904299 -0.292941 -0.543043  0.183428  ...   
4        0.945895  -1.105829 -0.292941 -0.543043  0.183428  ...   

   numOfWindowFeatures  lotSizeSqFt  livingAreaSqFt  avgSchoolDistance  \
0             1.620901    -0.851823        0.399818          -0.535249   
1            -0.426958    -0.732281       -0.422348          -0.410445   
2            -0.426958     0.404743       -0.708576          -0.597651   
3            -0.426958    -0.792052       -0.511177          -0.410445   
4            -0.426958    -0.372968       -0.063082          -0.660053   

   avgSchoolRating  avgSchoolSize  MedianStudentsPerTeacher  numOfBathrooms  \
0        -1.673595      -0.533854                 -0.492151        0.300632   
1        -1.673595      -0.533854                 -0.492151       -0.647765   
2        -1.494404      -0.396030                 -0.492151       -0.647765   
3        -1.673595      -0.533854                 -0.492151       -0.647765   
4        -0.956832      -0.043812                 -0.492151        0.300632   

   numOfBedrooms  numOfStories  
0       0.658444             2  
1       0.658444             1  
2      -0.518146             1  
3      -0.518146             1  
4      -0.518146             2  

[5 rows x 28 columns]

In [4]:
X = df.copy()
y = X.pop("latestPrice")
colnames_car = X.columns
colnames = X.columns

In [5]:
model = DecisionTreeRegressor()
model.fit(X, y)

DecisionTreeRegressor()

In [6]:
importance = model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
    
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

Feature: 0, Score: 0.09364
Feature: 1, Score: 0.07108
Feature: 2, Score: 0.03844
Feature: 3, Score: 0.00113
Feature: 4, Score: 0.00362
Feature: 5, Score: 0.01541
Feature: 6, Score: 0.00029
Feature: 7, Score: 0.00605
Feature: 8, Score: 0.00098
Feature: 9, Score: 0.00638
Feature: 10, Score: 0.00103
Feature: 11, Score: 0.03588
Feature: 12, Score: 0.01421
Feature: 13, Score: 0.00846
Feature: 14, Score: 0.00532
Feature: 15, Score: 0.00509
Feature: 16, Score: 0.00261
Feature: 17, Score: 0.00104
Feature: 18, Score: 0.00833
Feature: 19, Score: 0.51696
Feature: 20, Score: 0.01348
Feature: 21, Score: 0.04081
Feature: 22, Score: 0.03047
Feature: 23, Score: 0.00702
Feature: 24, Score: 0.06569
Feature: 25, Score: 0.00474
Feature: 26, Score: 0.00183


In [7]:
cart = DecisionTreeRegressor()
cart.fit(X,y)
ranks["CART"] = ranking(cart.feature_importances_, colnames);
ranks

{'CART': {'zipcode': 0.18,
  'latitude': 0.08,
  'longitude': 0.08,
  'propertyTaxRate': 0.0,
  'garageSpaces': 0.0,
  'hasAssociation': 0.03,
  'hasGarage': 0.0,
  'hasSpa': 0.0,
  'hasView': 0.0,
  'homeType': 0.01,
  'parkingSpaces': 0.0,
  'yearBuilt': 0.09,
  'numPriceChanges': 0.03,
  'numOfAppliances': 0.07,
  'numOfParkingFeatures': 0.01,
  'numOfPatioAndPorchFeatures': 0.01,
  'numOfSecurityFeatures': 0.01,
  'numOfWindowFeatures': 0.0,
  'lotSizeSqFt': 0.01,
  'livingAreaSqFt': 1.0,
  'avgSchoolDistance': 0.03,
  'avgSchoolRating': 0.08,
  'avgSchoolSize': 0.06,
  'MedianStudentsPerTeacher': 0.01,
  'numOfBathrooms': 0.13,
  'numOfBedrooms': 0.01,
  'numOfStories': 0.0}}

In [8]:
decimalrank_df = pd.DataFrame()

# Join preranks in dataframe
for method_name, rank_list in ranks.items():
    df_Temp = pd.DataFrame(data = {method_name:pd.Series(rank_list)})
    decimalrank_df = pd.concat([decimalrank_df,df_Temp ], axis=1)

rank_df = pd.DataFrame()
# Create new integer ranks
for method_name, rank_list in ranks.items():
    rank_df[method_name] = decimalrank_df[method_name].rank(ascending=False)

rank_df['feature_name'] = rank_df.index


In [9]:
ranksout = rank_df.melt(id_vars = 'feature_name', var_name='method', value_name='feature_rank')
ranksout['dataset'] = 'austin_housing'
ranksout

feature_name method  feature_rank         dataset
0                      zipcode   CART           2.0  austin_housing
1                     latitude   CART           6.0  austin_housing
2                    longitude   CART           6.0  austin_housing
3              propertyTaxRate   CART          23.5  austin_housing
4                 garageSpaces   CART          23.5  austin_housing
5               hasAssociation   CART          11.0  austin_housing
6                    hasGarage   CART          23.5  austin_housing
7                       hasSpa   CART          23.5  austin_housing
8                      hasView   CART          23.5  austin_housing
9                     homeType   CART          16.0  austin_housing
10               parkingSpaces   CART          23.5  austin_housing
11                   yearBuilt   CART           4.0  austin_housing
12             numPriceChanges   CART          11.0  austin_housing
13             numOfAppliances   CART           8.0  austin_housing
14        numOfParkingFeatures   CART          16.0  austin_housing
15  numOfPatioAndPorchFeatures   CART          16.0  austin_housing
16       numOfSecurityFeatures   CART          16.0  austin_housing
17         numOfWindowFeatures   CART          23.5  austin_housing
18                 lotSizeSqFt   CART          16.0  austin_housing
19              livingAreaSqFt   CART           1.0  austin_housing
20           avgSchoolDistance   CART          11.0  austin_housing
21             avgSchoolRating   CART           6.0  austin_housing
22               avgSchoolSize   CART           9.0  austin_housing
23    MedianStudentsPerTeacher   CART          16.0  austin_housing
24              numOfBathrooms   CART           3.0  austin_housing
25               numOfBedrooms   CART          16.0  austin_housing
26                numOfStories   CART          23.5  austin_housing

In [10]:
ranksout.to_csv('../data/CART/austin_housing_featureranks_CART.csv', sep=';',index=False)